<h1> The Battle Of Neighborhoods. Which neighborhoods in the Stockholm area are better suited for relocation given activities of interest and apartment prices</h1>

## Table of Contents
1. <a href="#intro"> Introduction / Background </a> 
2. <a href="#data"> Data</a> 

<div id="intro"> <h2>Introduction / Background </h2> </div>

<h3>1.1 Background</h3>

Although Sweden is quite an unpulated country, with around 10 million people overall, The country is quite extensive in area. The metropolitan area of Stockholm is the largest metropolitan area in Sweden and can be compared in size with cities like Paris, Barcelona and Milan. This means that there are over 150 neighborhoods where people can establish residence and go around their daily routines. Having lived in Stockholm for more than 5 years in several of this neighborhoods, i have noticed 4 main things that Swedish people, in general, really enjoy doing. The number one is to 'Fika'. Fika is a concept and is immerse within the culture of the country. It means to gather up with friends or colleagues for a coffe or tea and something small to eat, like a bun or a cake. The number two is outdoor activities such as hiking, skiing or bathing in the lakes. Being an archipelago, Stockholm is fully surrounded by nature which is highly appreciated it by locals, specially in the summer. Number three is to drink and dance. Although this is specially true for younger generations, people from all ages enjoy a lot having a drink in their local pubs, and are starting to get more attracted to local breweries around Stockholm. The number four is eating out in restaurants around the neighborhood. Whether is a lazy weekend day with no cooking, a socializing event or a curiosity for new gastronomy, locals like to eat out. 

<h3>1.2 Problem description</h3>

Part of the criteria for people in stockholm to move to a specific neighborhood, appart from accomodation prices, is the type of activities one can do around the neighborhood. Can you grab a cup of coffee nearby with a friend? is there nature around where you could go for a swim in the summer and maybe be a litle bit isolated from people? Is it possible to go drinking with colleagues and be able to walk back home? Or is there a neighborhood offering all of these posibilities? To answer these questions one could of course manually investigate each neighborhood, visit them or check online what activities are available around the localities. Given Stockholm's vast area and the big amount of neighborhoods, this would be a tedious and time consuming process. Instead, one could automatically create a sort of profile for each neighborhood, group them, and make this information available and visualizable for people to decide which options are more suitable. <br>
This project aims at identifying clusters of similar neighnorhoods in Stockholm taking into account the availability to carry on one or more of swedes preferred activities (generally speaking), namely: to Fika, outdoor activities, to drink/party and to eat out in restaurants. Additionally, to constrast these clusters with the average municipality accomodation prices. By doing so, the goal is to help people living in Stockholm to identify which neigborhood better match their profiles, and which ones would be a better option for them if they decide to relocate within Stockholm.

<div id="data"> <h2>Data Collection</h2> </div>

Sweden's territory is divided into Counties. Counties itself are divided into municipalities / Boroughs or what is known in Sweden as 'Komuns'. If one zoom in even further we find that each komun is divided into several districts or neigborhoods. In this project, data from neighborhoods within Stockholm county will be considered.

<h3>1.1 Stockholm Komuns (Municipalities or Boroughs)</h3>

Stockholm has a total of 26 komuns. <br> 
Since we want to analyze neigborhoods, we need to obtain which are Stockholm's Komuns so we can later obtain its corresponding neigborhoods.<br> 
Information about Sweden's Counties and Komuns is available to download in the web for free. <br> 
The following repo provides both County level and Municipality level information within Sweden. This information includes the geographical limits and borders of each County / Municipilaty: https://github.com/deldersveld/topojson/tree/master/countries/sweden 

<h3>1.2 Stockholm Districs (Neighborhoods)</h3>

Once we have the name of the Komuns, we need to obtain their corresponding districts. There is no open dataset in the web containing this information. So the procedure to obtain this data data was to scrape the information from wikipedia.<br>
If we look at the main wikipedia page for Stockholm County: https://en.wikipedia.org/wiki/Stockholm_County we can observe a table with all Komun names, plus a link to its corresponding wikipedia page. These individual pages of each Komun will be used to obtain the corresponding neighborhoods. Additionally, Google's Geolocation API was used to obtain the center location point of each gathered neighborhood.

<h3>1.3 Venue data</h3>

The Foursquare API will be used to obtain information related to the venues around a given neigborhood.

<h4>1.3.1 Explore API</h4>

The explore endpoint of the Foursquare API is used to obtain venue categories related to a specified location and query. By doing this, we can obtain all related venue categories to a specific activity such as 'Fika'. Consequently, we can group together all venues that belong to the found categories and have rough idea of the weight of 'Fika' venues around the neigborhood area.

<h4>1.3.2 Search API</h4>

The search endpoint of the Foursquare API is used to obtain all venues around a neighborhood within a certain radio range. <br>
NOTE: Using the free version of Foursquare API there is a limitation of a 100 venues that can be obtained in a given query

<h3>1.4 House prices</h3>

In order to obtain average housing prices for each Komun, the main real state website for searching apartments will be used. The website is called www.hemnet.se. Hemnet provides an API to make queries and requests. However, given that it was a litle cumbersome to set it up and there is a time limitation for the project, the data will scrapped instead from URLs. This is not the most elegant approach, but is quite fast to set up and obtain the desired results. 

In [2]:
# imports
import numpy as np
import pandas as pd